# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *
from create_tables import create_database

In [2]:
create_database()

database "sparkifydb" already exists



(<cursor object at 0x7f2ac670f618; closed: 0>,
 <connection object at 0x7f2ac60713d8; dsn: 'user=student password=xxx dbname=studentdb host=127.0.0.1', closed: 0>)

In [3]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()
conn.set_session(autocommit=True)

In [4]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [5]:
song_files = get_files("data/song_data/")

In [6]:
filepath = song_files

In [7]:
# df_check = pd.read_json(filepath, lines=True)
# df_check.head()

In [8]:
li = []

for path_ in song_files:
    df_ = pd.read_json(path_, lines=True)
    li.append(df_)

df = pd.concat(li, axis=0, ignore_index=True)

In [9]:
df.shape

(71, 10)

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [10]:
df.columns

Index(['artist_id', 'artist_latitude', 'artist_location', 'artist_longitude',
       'artist_name', 'duration', 'num_songs', 'song_id', 'title', 'year'],
      dtype='object')

In [11]:
song_data = df[[ "song_id", "title", "artist_id", "year", "duration"]].values
tuple(song_data[0])

('SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, 218.93179)

In [12]:
# songs_table_drop = ("""DROP TABLE songs;""")
# cur.execute(songplay_table_create)

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [13]:
df.columns

Index(['artist_id', 'artist_latitude', 'artist_location', 'artist_longitude',
       'artist_name', 'duration', 'num_songs', 'song_id', 'title', 'year'],
      dtype='object')

In [14]:

song_table = ("""CREATE TABLE IF NOT EXISTS songs(
song_id TEXT,
title TEXT,
artist_id TEXT,
year INT,
duration float);""")
cur.execute(song_table)


In [15]:
# song_data_[0][0]

In [16]:
song_table_insert = ("INSERT INTO songs (song_id, title, artist_id, year, duration) VALUES (%s, %s, %s, %s, %s)")
# cur.execute("INSERT INTO song_length (song_id, song_length) VALUES (%s, %s)", \
#                  (5, 227))


In [17]:
 song_data_ = tuple([list(song) for song in song_data])

In [18]:
mm = song_data_[0][0]

In [20]:
# for data in song_data_:
#     cur.execute(song_table_insert, data)

In [21]:
# df.head()

In [22]:
# for col in ["artist_id", "num_songs",  "year"]:
#     df[col] = convert_to_dtype(col, int)

In [23]:
# df.to_sql(df, conn, if_exists='append', method="multi")


Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [ ]:
df.columns

In [ ]:
artist_data = df[[ "artist_id", "artist_name", "artist_location", "artist_latitude", "artist_longitude"]].values
artist_data[0]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [ ]:
artist_table_create = ("""CREATE TABLE IF NOT EXISTS artists (artist_id TEXT, name TEXT, location TEXT, latitude FLOAT, longitude FLOAT);""")
cur.execute(artist_table_create)

In [ ]:
artist_table_insert = ("INSERT INTO artists (artist_id, name, location, latitude, longitude) VALUES (%s, %s, %s, %s, %s)")

In [ ]:
 artist_data_ = tuple([list(song) for song in artist_data])

In [ ]:
artist_data_ = tuple([list(song) for song in artist_data])
for data in artist_data_:
#     cur.execute(song_table_insert, data)
    cur.execute(artist_table_insert, data)
# conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [ ]:
log_files = get_files("data/log_data/")

In [ ]:
filepath = log_files

In [ ]:
li = []

for path_ in filepath:
    df_ = pd.read_json(path_, lines=True)
    li.append(df_)

df = pd.concat(li, axis=0, ignore_index=True)

In [ ]:
# df = pd.read_json(file_path, lines=True)
df.head()

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [ ]:
df = df.loc[df["page"] == 'NextSong']
df.head()

In [ ]:
df["ts"] = pd.to_datetime(df["ts"], unit="ms")

In [ ]:
t = pd.to_datetime(df["ts"], unit="ms")
t.head()

In [ ]:
column_labels = ["time", "hour", "day", "week_of_year", "month", "year", "weekday"]

In [ ]:
time_df = pd.DataFrame([], columns=column_labels)


In [ ]:
time_table_create = ("""CREATE TABLE IF NOT EXISTS time (start_time timestamp, hour int, day int, week int, month int, year int, weekday int);""")
cur.execute(time_table_create)

In [ ]:
time_df["time"] = t
time_df["hour"] = t.dt.hour

time_df["day"] = t.dt.day
time_df["week_of_year"] = t.dt.weekofyear
time_df["month"] = t.dt.month
time_df["year"] = t.dt.year
time_df["weekday"] = t.dt.weekday

In [ ]:
# time_data = [t.dt.hour, t.dt.day, t.dt.weekofyear, t.dt.month, t.dt.year, t.dt.weekday]


In [ ]:
time_df.head()

In [ ]:
time_table_insert = ("INSERT INTO time (start_time, hour, day, week, month, year, weekday) VALUES (%s, %s, %s, %s, %s,  %s, %s)")

In [ ]:
# time_data

In [ ]:
time_data_ = tuple([list(tt) for tt in time_df.values])
for data in time_data_:
    cur.execute(time_table_insert, data)


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [ ]:
# for i, row in time_df.iterrows():
#     cur.execute(time_table_insert, list(row))
#     conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [ ]:
df.head()

In [ ]:
users_table_create = ("""CREATE TABLE IF NOT EXISTS users (user_id INT, first_name TEXT, last_name TEXT, gender TEXT, level TEXT);""")
cur.execute(users_table_create)


In [ ]:
users_table_insert = ("INSERT INTO users (user_id, first_name, last_name, gender, level) VALUES (%s, %s, %s, %s, %s)")

In [ ]:
# df.head()

In [ ]:
user_df = df[["userId", "firstName", "lastName", "gender", "level"]]

In [ ]:
user_data_ = tuple([list(tt) for tt in user_df.values])
for data in user_data_:
    cur.execute(users_table_insert, data)

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [ ]:
# for i, row in user_df.iterrows():
#     cur.execute(user_table_insert, row)
#     conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [ ]:
# df.columns




In [ ]:
# songsplays_df = df[[ "ts", "userId", "level", "song_id", "artist_id", "session_id", "location", "user_agent"]]

In [ ]:
# song_select = ("""SELECT * FROM songs;""")


In [ ]:
# songplay_id,
# start_time,
# user_id,
# level,
# song_id,
# artist_id, session_id, location, user_agent


In [ ]:

# songplay_table_create =("""DROP TABLE songplays""")
# cur.execute(songplay_table_create)

In [ ]:

songplay_table_create =("""CREATE TABLE IF NOT EXISTS songplays (songplay_id TEXT, start_time TIMESTAMP,
user_id TEXT, level TEXT, song_id TEXT, artist_id TEXT, session_id TEXT, location TEXT, user_agent TEXT)""")
cur.execute(songplay_table_create)

In [ ]:
# df.artist
song_select = ("""SELECT song_id, a.artist_id FROM songs s LEFT JOIN artists a on a.artist_id=s.artist_id WHERE s.title =%s AND a.name=%s ;""")


In [ ]:
songplay_table_insert = ("""INSERT INTO songplays (songplay_id, start_time, user_id, level, song_id, artist_id, session_id, location, user_agent) VALUES (%s, %s, %s, %s, %s, %s,  %s, %s, %s)""")


In [ ]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
#     print(row.song,"-",row.artist )
    cur.execute(song_select, (row.song, row.artist ))
    results = cur.fetchone()
    
    if results:
        print(results)
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (index, row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent )
    cur.execute(songplay_table_insert, songplay_data) 
#     conn.commit()

In [ ]:
# df.loc[df["artist"] != 'Faiz Ali Faiz']
len(df)

In [ ]:
# artist_data_

Run `test.ipynb` to see if you've successfully added records to this table.

In [24]:
songplay_table_drop = "DROP TABLE songplays"
user_table_drop = "DROP TABLE users"
song_table_drop = "DROP TABLE songs"
artist_table_drop = "DROP TABLE artists"
time_table_drop = "DROP TABLE time"


In [44]:
cur.execute(songplay_table_drop) 
cur.execute(user_table_drop) 
cur.execute(artist_table_drop) 
cur.execute(song_table_drop) 
cur.execute(time_table_drop)

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.

In [ ]:
print("DONE")